Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import math

Reading the AIS data and Extracting Latitude, Longitude, and Time Columns

In [ ]:
# Read CSV file into DataFrame
df = pd.read_csv(r"/content/209030000 J31.csv")

# Extract latitude, longitude, and time columns
latitudes = df["LAT"].values
longitudes = df["LON"].values
times = pd.to_datetime(df["BaseDateTime"], format="%Y-%m-%dT%H:%M:%S")

Vincenty Formula

In [ ]:
def vincenty(lat1, lon1, lat2, lon2):
    # WGS-84 ellipsiod parameters
    a = 6378137.0  # semi-major axis in meters
    f = 1 / 298.257223563  # flattening

    # Convert latitude and longitude from degrees to radians
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    lambda1 = math.radians(lon1)
    lambda2 = math.radians(lon2)

    # U1, U2 and LAMBDA are intermediates for later calculations
    U1 = math.atan((1 - f) * math.tan(phi1))
    U2 = math.atan((1 - f) * math.tan(phi2))
    LAMBDA = lambda2 - lambda1

    # Initial guess for sigma (angular distance between points)
    sigma = 0

    # Iteratively solve for sigma until change is negligible
    for _ in range(1000):  # max iterations set to 1000
        sin_sigma = math.sqrt((math.cos(U2) * math.sin(LAMBDA)) ** 2 +
                              (math.cos(U1) * math.sin(U2) - math.sin(U1) * math.cos(U2) * math.cos(LAMBDA)) ** 2)
        cos_sigma = math.sin(U1) * math.sin(U2) + math.cos(U1) * math.cos(U2) * math.cos(LAMBDA)
        sigma_prev = sigma
        sigma = math.atan2(sin_sigma, cos_sigma)
        if abs(sigma - sigma_prev) < 1e-12:  # convergence criterion
            break

    # Calculate the distance
    A = (math.cos(U2) * math.sin(LAMBDA)) ** 2 + (math.cos(U1) * math.sin(U2) -
                                                   math.sin(U1) * math.cos(U2) * math.cos(LAMBDA)) ** 2
    B = math.sin(U1) * math.sin(U2) + math.cos(U1) * math.cos(U2) * math.cos(LAMBDA)
    delta_sigma = f / 16 * A * (4 + f * (4 - 3 * A))
    s = a * (sigma - delta_sigma)

    return s

Calculating Distances Between Consecutive Points

In [ ]:
# Calculate distances between consecutive points
total_distance = 0.0
for i in range(1, len(latitudes)):
    lat1, lon1 = latitudes[i - 1], longitudes[i - 1]
    lat2, lon2 = latitudes[i], longitudes[i]
    distance = vincenty(lat1, lon1, lat2, lon2)
    total_distance += distance
    distancekm = total_distance/1000

print("Total distance traveled by the ship:", distancekm, "kilometers")

Total distance traveled by the ship: 1040.3067632689995 kilometers


Calculating Total Time Elapsed

In [ ]:
# Calculate total time elapsed
total_time = times.iloc[-1] - times.iloc[0]  # Assuming the timestamps are sorted
total_time_seconds = total_time.total_seconds()

Calculating Average Speed

In [ ]:
# Calculate average speed
average_speed = total_distance / total_time_seconds  # in meters per second

# Convert average speed to kilometers per hour (optional)
average_speed_kmh = average_speed * 3.6  # 1 m/s = 3.6 km/h

print("Average speed of the ship:", average_speed_kmh, "kilometers per hour")

Average speed of the ship: -18.52727984450578 kilometers per hour


Starting Time of the Voyage

In [ ]:
# Calculate starting time
start_time = times.iloc[0]  # Get the timestamp of the first recorded data point

print("Starting Time:", start_time)

Starting Time: 2018-07-21 04:44:00


Calculating Estimated Time of Arrival (ETA)

In [ ]:
# Calculate ETA
# Assume the ship is currently at the last recorded data point in the dataset
current_time = times.iloc[-1]  # Get the timestamp of the last recorded data point

# Calculate time required to cover the total distance at the average speed
eta_seconds = total_distance / average_speed

# Calculate ETA by adding the time required to the current time
eta_time = current_time + pd.Timedelta(seconds=eta_seconds)

print("Estimated Time of Arrival (ETA):", eta_time)

Estimated Time of Arrival (ETA): 2018-07-16 12:26:00


In [ ]:
# View the first row
print("First row:")
print(df.head(1))

# View the last row
print("\nLast row:")
print(df.tail(1))

print("\nAverage speed of the ship:", average_speed_kmh, "kilometers per hour")

print("Total distance traveled by the ship:", distancekm, "kilometers")

print("\nEstimated Time of Arrival (ETA):", eta_time)

First row:
        MMSI         BaseDateTime       LAT       LON  SOG    COG  Heading  \
0  209030000  2018-07-21T04:44:00  58.38182  24.47772  0.0  206.0      206   

   VesselName  IMO  CallSign  VesselType  Status  Length  Width  Draft  Cargo  \
0         NaN  NaN       NaN          70     NaN     NaN     15    5.3     70   

  TransceiverClass  
0               IB  

Last row:
          MMSI         BaseDateTime       LAT       LON  SOG    COG  Heading  \
285  209030000  2018-07-18T20:35:00  54.36555  10.14277  0.0  105.0      105   

     VesselName  IMO  CallSign  VesselType  Status  Length  Width  Draft  \
285         NaN  NaN       NaN          70     NaN     NaN     15    5.3   

     Cargo TransceiverClass  
285     70               IB  

Average speed of the ship: -18.52727984450578 kilometers per hour
Total distance traveled by the ship: 1040.3067632689995 kilometers

Estimated Time of Arrival (ETA): 2018-07-16 12:26:00


Source

In [ ]:
# https://nathanrooy.github.io/posts/2016-12-18/vincenty-formula-with-python/
# https://pypi.org/project/vincenty/ For 2 Cities, we do for 2 Consecutive Points
# https://www.johndcook.com/blog/2018/11/24/spheroid-distance/
# https://codereview.stackexchange.com/questions/274709/vincentys-distance-direct-formulae-numpy